In [2]:
import openai
import pandas as pd
import numpy as np
import os
from openai.embeddings_utils import cosine_similarity
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter

### Procesar datos de un PDF

In [1]:
loader = PyPDFLoader("codigoPenal.pdf")
pages = loader.load_and_split()

NameError: name 'PyPDFLoader' is not defined

In [4]:
pages[1].page_content

'Responsabilidad Penal  \nArtículo VII. - La pena requiere de la responsabilidad penal del autor. Queda proscrita \ntoda forma de responsabilidad objetiva.  \nProporcionalidad de las sanciones  \nArtículo VIII. - La pena no puede sobrepasar la responsabilidad por el hecho. Esta \nnorma no rige en caso de reincidencia ni de habitualidad del agente al delito. La \nmedida de seguridad sólo puede ser ordenada por intereses públicos predominantes.  \nFines de la Pena y Medidas de Seguridad  \nArtículo IX. - La pena tiene función preventiva, protectora y resocializadora. Las \nmedidas de seguridad persiguen fines de curación, tutela y rehabilitación.  \nAplicación Supletoria de la Ley Penal  \nArtículo X. - Las normas generales de este Código son aplicables a los hechos \npunibles previstos en leyes especiales.  \nLIBRO PRIMERO:  PARTE GENERAL  \nTÍTULO I: DE LA LEY PENAL  \nCAPÍTULO I: APLICACIÓN  ESPACIAL  \nArtículo 1. - Principio de Territorialidad  \nLa Ley Penal peruana se aplica a tod

In [5]:
len(pages)

226

In [6]:
split = CharacterTextSplitter(chunk_size=400, separator = '  \n')
texts = split.split_documents(pages)

Created a chunk of size 645, which is longer than the specified 400
Created a chunk of size 620, which is longer than the specified 400
Created a chunk of size 538, which is longer than the specified 400
Created a chunk of size 692, which is longer than the specified 400
Created a chunk of size 583, which is longer than the specified 400
Created a chunk of size 586, which is longer than the specified 400
Created a chunk of size 417, which is longer than the specified 400
Created a chunk of size 502, which is longer than the specified 400
Created a chunk of size 466, which is longer than the specified 400
Created a chunk of size 538, which is longer than the specified 400
Created a chunk of size 620, which is longer than the specified 400
Created a chunk of size 448, which is longer than the specified 400
Created a chunk of size 506, which is longer than the specified 400
Created a chunk of size 727, which is longer than the specified 400
Created a chunk of size 485, which is longer tha

In [7]:
texts[2].page_content

'Prohibición de la Analogía  \nArtículo III. - No es permitida la analogía para calificar el hecho como delito o falta, \ndefinir un estado de peligrosidad o determinar la pena o medida de seguridad que les \ncorresponde.  \nPrincipio de Lesividad  \nArtículo IV. - La pena, necesariamente, precisa de la lesión o puesta en peligro de \nbienes jurídicos tutelados por la ley.  \nGarantía Jurisdiccional'

In [8]:
len(texts)

1483

In [9]:
texts = [str(text.page_content) for text in texts] #Lista de parrafos
paragraphs = pd.DataFrame(texts, columns=["text"])

In [10]:
paragraphs

,text
0,CÓDIGO PENAL \nDECRETO LEGISLATIVO 635 \nTít...
1,Finalidad Preventiva \nArtículo I. - Este Cód...
2,Prohibición de la Analogía \nArtículo III. - ...
3,Principio de Lesividad \nArtículo IV. - La pe...
4,Artículo V. - Sólo el Juez competente puede im...
...,...
1478,1. Las sentencias a pena de internamiento con ...
1479,2. Las sentencias a penas de p risión o penite...
1480,4. Las sentencias a penas de inhabilitación de...
1481,"5. Las sentencias a pena de multa, fijada conf..."


In [11]:
from sentence_transformers import SentenceTransformer

d:\cursos UNI\cursos 23-1\AnaliticaDeDatos\TGrupales\FINAL\JuezBot\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

In [14]:
paragraphs['Embedding'] = paragraphs["text"].apply(lambda x: model.encode(x)) # Nueva columna con los embeddings de los parrafos
paragraphs.to_csv('embeddings.csv')

In [15]:
paragraphs

,text,Embedding
0,CÓDIGO PENAL \nDECRETO LEGISLATIVO 635 \nTít...,"[0.010199041, 0.23420632, -0.19662668, 0.05728..."
1,Finalidad Preventiva \nArtículo I. - Este Cód...,"[0.056329608, 0.18835348, -0.24528345, 0.09495..."
2,Prohibición de la Analogía \nArtículo III. - ...,"[0.085120454, -0.00930684, 0.00066791364, 0.05..."
3,Principio de Lesividad \nArtículo IV. - La pe...,"[0.074587435, 0.08403673, -0.054432847, -0.052..."
4,Artículo V. - Sólo el Juez competente puede im...,"[0.0057146926, 0.02657123, -0.016760958, -0.04..."
...,...,...
1478,1. Las sentencias a pena de internamiento con ...,"[-0.0030800109, 0.16682257, 0.0611269, -0.0610..."
1479,2. Las sentencias a penas de p risión o penite...,"[-0.037034243, 0.08793425, 0.016187878, -0.018..."
1480,4. Las sentencias a penas de inhabilitación de...,"[0.08741069, 0.15282379, 0.116064094, -0.00689..."
1481,"5. Las sentencias a pena de multa, fijada conf...","[0.011887475, 0.087663144, 0.04723517, -0.0859..."


In [116]:
knowledge_base = pd.read_csv('embeddings.csv')

In [117]:
knowledge_base = knowledge_base.drop(knowledge_base.columns[0], axis=1)
knowledge_base

,text,Embedding
0,CÓDIGO PENAL \nDECRETO LEGISLATIVO 635 \nTít...,[ 1.01990411e-02 2.34206319e-01 -1.96626678e-...
1,Finalidad Preventiva \nArtículo I. - Este Cód...,[ 5.63296080e-02 1.88353479e-01 -2.45283455e-...
2,Prohibición de la Analogía \nArtículo III. - ...,[ 0.08512045 -0.00930684 0.00066791 0.053062...
3,Principio de Lesividad \nArtículo IV. - La pe...,[ 7.45874345e-02 8.40367302e-02 -5.44328466e-...
4,Artículo V. - Sólo el Juez competente puede im...,[ 5.71469264e-03 2.65712291e-02 -1.67609584e-...
...,...,...
1478,1. Las sentencias a pena de internamiento con ...,[-3.08001088e-03 1.66822568e-01 6.11268990e-...
1479,2. Las sentencias a penas de p risión o penite...,[-0.03703424 0.08793425 0.01618788 -0.018388...
1480,4. Las sentencias a penas de inhabilitación de...,[ 8.74106884e-02 1.52823791e-01 1.16064094e-...
1481,"5. Las sentencias a pena de multa, fijada conf...",[ 0.01188747 0.08766314 0.04723517 -0.085938...


In [105]:
knowledge_base.dtypes

text         object
Embedding    object
dtype: object

In [118]:
print(knowledge_base.iloc[0,1])

[ 1.01990411e-02  2.34206319e-01 -1.96626678e-01  5.72853610e-02
  2.35024005e-01  1.48634419e-01  1.69670396e-02  1.31847754e-01
 -1.02606751e-02  2.21784160e-01  2.09920004e-01  1.14301696e-01
  5.95639125e-02 -1.63720027e-02 -3.63679901e-02 -5.40374182e-02
 -5.62863424e-02  2.24696975e-02  1.17049869e-02  1.03825696e-01
  1.92531094e-01  1.00697763e-01  1.25380874e-01 -5.19504920e-02
 -2.54078865e-01  7.13783363e-03 -1.21948518e-01 -1.57919914e-01
  5.34923077e-02 -2.44170185e-02 -1.61913469e-01 -6.63562268e-02
  1.22049026e-01  1.09952956e-01 -1.18728317e-01 -1.43067583e-01
  1.94878340e-01  9.47726071e-02  3.20736840e-02  4.98162359e-02
 -1.44885510e-01  3.91587429e-02 -2.46980805e-02  5.91718443e-02
 -2.12066323e-02  2.65908195e-03  2.41010990e-02 -4.06131223e-02
 -1.53981194e-01 -2.07470238e-01 -3.38788517e-02  2.12621093e-02
  1.76056586e-02  8.27829018e-02 -1.27735632e-02 -2.52171338e-01
  9.38376561e-02 -4.52240650e-03 -5.64309210e-02  8.38433877e-02
  1.28850192e-01 -4.64122

In [119]:
knowledge_base['Embedding'] = knowledge_base['Embedding'].str.strip('[]').str.split()
knowledge_base['Embedding'] = knowledge_base['Embedding'].apply(lambda x: [float(num) for num in x])
knowledge_base

,text,Embedding
0,CÓDIGO PENAL \nDECRETO LEGISLATIVO 635 \nTít...,"[0.0101990411, 0.234206319, -0.196626678, 0.05..."
1,Finalidad Preventiva \nArtículo I. - Este Cód...,"[0.056329608, 0.188353479, -0.245283455, 0.094..."
2,Prohibición de la Analogía \nArtículo III. - ...,"[0.08512045, -0.00930684, 0.00066791, 0.053062..."
3,Principio de Lesividad \nArtículo IV. - La pe...,"[0.0745874345, 0.0840367302, -0.0544328466, -0..."
4,Artículo V. - Sólo el Juez competente puede im...,"[0.00571469264, 0.0265712291, -0.0167609584, -..."
...,...,...
1478,1. Las sentencias a pena de internamiento con ...,"[-0.00308001088, 0.166822568, 0.061126899, -0...."
1479,2. Las sentencias a penas de p risión o penite...,"[-0.03703424, 0.08793425, 0.01618788, -0.01838..."
1480,4. Las sentencias a penas de inhabilitación de...,"[0.0874106884, 0.152823791, 0.116064094, -0.00..."
1481,"5. Las sentencias a pena de multa, fijada conf...","[0.01188747, 0.08766314, 0.04723517, -0.085938..."


In [120]:
type(knowledge_base.iloc[0,1])

list

In [121]:
type(knowledge_base.iloc[0,1][0])

float

In [124]:
question = '¿la pena por homicidio calificado?'

In [125]:
question_embed = model.encode(question)

In [126]:
print(question_embed)

[ 0.21032871  0.0856709  -0.15632981  0.01335955  0.5094658   0.04569569
  0.35193023  0.01303518  0.10874153  0.389913    0.27479884 -0.0846829
 -0.13101439 -0.01061767 -0.22028618 -0.18735676 -0.04222305 -0.25545797
 -0.25266287  0.18770657 -0.01024128  0.13728175  0.539918   -0.07049277
  0.11170628 -0.15195996 -0.03477295  0.04474377 -0.23344225  0.01910379
 -0.07731687 -0.15689911 -0.17022742 -0.11608572 -0.00827368 -0.17192997
  0.28422686  0.31276977 -0.18069905 -0.25579238 -0.21982206  0.46936607
 -0.12158446  0.06953526  0.05796552 -0.10172577 -0.23064908  0.17407924
 -0.29328397 -0.20201904  0.03387491  0.09157315 -0.09443047  0.24598256
  0.08355835 -0.47563866 -0.07482043  0.1036382  -0.10870533  0.23940724
 -0.21958093 -0.08960059  0.06706493 -0.18011966  0.04764018 -0.13771719
  0.08665052 -0.16421889 -0.0238576   0.5006388   0.1630622   0.0299745
  0.13385221 -0.13501847 -0.21445361 -0.23651218 -0.02328429  0.12781417
  0.24341393 -0.25426856  0.11408689  0.0278082   0.0

In [127]:
knowledge_base['Similarity'] = knowledge_base["Embedding"].apply(lambda x: cosine_similarity(x, question_embed))
knowledge_base = knowledge_base.sort_values('Similarity', ascending=False)


In [128]:
knowledge_base

,text,Embedding,Similarity
309,"Artículo 112. - Homicidio piadoso \nEl que, p...","[0.0434350967, 0.0768171921, -0.121408165, -0....",0.750230
285,Artículo 108. - Homici dio calificado \nSerá ...,"[0.00616669, 0.2675636, -0.13731667, -0.088770...",0.728749
306,"Artículo 111. - Homicidio Culposo \nEl que, p...","[-0.0193109307, 0.121225983, -0.130721897, -0....",0.715270
303,Artículo 1 09.- Homicidio por emoción violenta...,"[0.0125552472, 0.137540042, -0.101905376, 0.15...",0.708614
286,"1. Por ferocidad, codicia, lucro o por placer....","[-0.032903716, 0.134831339, -0.176840395, 0.12...",0.698871
...,...,...,...
814,e. Un esquema de trazado (tipografía) registra...,"[-0.182207912, -0.11940182, -0.0909199566, -0....",0.012564
1407,Es funcionario o servidor público extranjero t...,"[-0.191319853, -0.105668768, 0.0957769305, -0....",0.008402
694,5. Si se realiza en el comercio de bienes mueb...,"[0.0609379373, 0.0370237418, 0.0566918068, -0....",0.004114
660,Artículo 186 -A.- Dispositivos para asistir a ...,"[-0.0821218491, 0.137757972, -0.155242577, -0....",-0.004578


In [130]:
knowledge_base.iloc[1,0]

'Artículo 108. - Homici dio calificado  \nSerá reprimido con pena privativa de libertad no menor de quince años el que mate a \notro concurriendo cualquiera de las circunstancias siguientes:  \n1. Por ferocidad, codicia, lucro o por placer.  \n2. Para facilitar u ocultar otro delito.  \n3. Con gr an crueldad o alevosía.'

In [132]:
select_paragraphs = knowledge_base.head(3)

In [133]:
select_paragraphs.head(3)

,text,Embedding,Similarity
309,"Artículo 112. - Homicidio piadoso \nEl que, p...","[0.0434350967, 0.0768171921, -0.121408165, -0....",0.750230
285,Artículo 108. - Homici dio calificado \nSerá ...,"[0.00616669, 0.2675636, -0.13731667, -0.088770...",0.728749
306,"Artículo 111. - Homicidio Culposo \nEl que, p...","[-0.0193109307, 0.121225983, -0.130721897, -0....",0.715270


In [134]:
context = ' '.join(select_paragraphs['text'])

In [135]:
context

'Artículo 112. - Homicidio piadoso  \nEl que, por piedad, mata a un enfermo incurable que le solicita de manera expresa y \nconsciente para poner fin a sus intolerables dolores, será reprimido con pena \nprivativa de libertad no mayor de tres años. Artículo 108. - Homici dio calificado  \nSerá reprimido con pena privativa de libertad no menor de quince años el que mate a \notro concurriendo cualquiera de las circunstancias siguientes:  \n1. Por ferocidad, codicia, lucro o por placer.  \n2. Para facilitar u ocultar otro delito.  \n3. Con gr an crueldad o alevosía. Artículo 111. - Homicidio Culposo  \nEl que, por culpa, ocasiona la muerte de una persona, será reprimido con pena \nprivativa de libertad no mayor de dos años o con prestación de servicios comunitarios \nde cincuenta y dos a ciento cuatro jornadas.'

In [136]:
def enviar_prompt(prompt, engine="text-davinci-002", temp=0.5, max_tokens=100, top_p=1, frequency_penalty=0, presence_penalty=0):
    respuesta = openai.Completion.create(
                                            engine=engine,
                                            prompt=prompt,
                                            temperature=temp,
                                            max_tokens=max_tokens,
                                            top_p=top_p,
                                            frequency_penalty=frequency_penalty,
                                            presence_penalty=presence_penalty
                                            )
    return respuesta['choices'][0]['text']

In [138]:
respuesta = enviar_prompt(
    "en base al siguiente contexto: "+context+"\nresponde a la siguiente pregunta: "+question
)


print(respuesta)



La pena por homicidio calificado es de quince años de prisión.
